In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()


def open_heic(file_path):
    image = Image.open(file_path)
    # Convert the PIL image to a NumPy array
    opencv_image = np.array(image)    
    return opencv_image

In [2]:
dataset_path = "dataset_texturas_v2_test"
images_path = (
    glob.glob(f"{dataset_path}/*/*.png")
    + glob.glob(f"{dataset_path}/*/*.jpeg")
    + glob.glob(f"{dataset_path}/*/*.jpg")
    + glob.glob(f"{dataset_path}/*/*.heic")
)

In [3]:
def img_crop(img):
    h, w, c = img.shape

    if h < w:
        diff = (w - h) // 2
        new_image = img[:, diff : diff + h]
    else:
        diff = (h - w) // 2
        new_image = img[diff : diff + h, :]

    return new_image


for i, image_path in enumerate(images_path):
    img_name = image_path.split("\\")[-1]

    try:
        # Read the image
        if image_path.__contains__("HEIC"):
            img = open_heic(image_path)
        else:
            img = cv.imread(image_path)

        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    except Exception as e:
        print(f"An error occurred: {e}")
        print(image_path)

    # crop
    img_croped = img_crop(img=img)

    # two-step resize
    img_croped = cv.resize(img_croped, (1024, 1024), interpolation=cv.INTER_CUBIC)
    img_croped = cv.resize(img_croped, (256, 256), interpolation=cv.INTER_CUBIC)

    # convert color to save
    img_croped = cv.cvtColor(img_croped, cv.COLOR_RGB2BGR)

    # new image name and path
    write_image_path = image_path.replace(img_name, "").replace(
        "dataset_texturas", "resized_dataset_texturas"
    )

    # Create the output folder if it doesn't exist
    if not os.path.exists(write_image_path):
        os.makedirs(write_image_path)

    cv.imwrite(write_image_path + f"img_{i}.jpeg", img_croped)

In [ ]:
img_path_ = r"dataset_texturas\acf_chap\IMG_7415.HEIC"

img = open_heic(img_path_)
print(img.shape)
plt.imshow(img)
plt.show()